# Collaborative Filtering on implicit feedback dataset

## Libraries and packages
Usual numerical libraries are used. In particular:
- Numpy allows for better mathematical operations
- Scipy is mainly used to create sparse matrices
- Time is imported in order to track the time elapsed while running scripts
- Implicit contains fast methods to deal with recommending systems over implicit feedback datasets

In [1]:
import numpy as np
import scipy.sparse as sp
import time
import implicit

## Loading data

Loading function has been imported by the notebooks we've seen during classes: it allows to convert a .csv file (or .dat) in a list of tuples with the form $(userid, itemid, rating)$.

In [2]:
def load_data(filename):
    input_lines = []
    users = {}
    num_users = 0
    items = {}
    num_items = 0
    raw_lines = open(filename, 'r').read().splitlines()
    # remove the first line
    del raw_lines[0]
    for line in raw_lines:
        line_content = line.split('::')
        user_id = int(line_content[0])
        item_id = int(line_content[1])
        rating = float(line_content[2])
        if user_id not in users:
            users[user_id] = num_users
            num_users += 1
        if item_id not in items:
            items[item_id] = num_items
            num_items += 1
        input_lines.append([users[user_id], items[item_id], rating])
    return input_lines, num_users, num_items

In [3]:
# 1m Movielens dataset is the input file
input_file = "./ratings_1m.dat"
input_ratings, num_users, num_items = load_data(input_file)

## Preprocessing data

The list obtained above is processed in order to get the (sparse) `ratings` matrix. The entry $(i,j)$ of such a matrix is $1$ if $(i,j,x)$ is a tuple in the list obtained by loading the file above, $0$ otherwise. $x$ represents the rating of user i toward item j but since we're only analyzing implicit feedback we basically convert every non-zero rating as $1$. 

An explaination of why this is not necessarily an over-semplification of the problem is given in the report.

A `test` matrix is created too. For every row (user) it contains only one non-zero entry which represents the test interaction for that user that will be used in the evaluation part below.

Notice that sparse matrices not only allows to save memory but also allows to represent the matrix as a list of non-zero elements and their location. Non-sparse matrix *ratings* shouldn't be used when dealing with a massive dataset but in our case. 

In [12]:
ratings = sp.dok_matrix((num_users, num_items))
test = sp.dok_matrix((num_users, num_items))


# This cycle allows to put every interaction (user,item) in the ratings matrix except one for each user,
# which is used as test.
for i in range(len(input_ratings)):
        if i > 0 and input_ratings[i - 1][0] == input_ratings[i][0]:
            ratings[input_ratings[i][0], input_ratings[i][1]] = 1
        else:
            test[input_ratings[i][0], input_ratings[i][1]] = 1

# We transpose in order to obtain a matrix with dimensions (num_items, num_users) which
# is the standard format for the implicit library.
ratings = ratings.transpose()

## Model Creation

The NearestNeighbor model is created using the library `implicit`.
Different classes can be used and they differs in how similarity is calculated.
For example `ItemItemRecommender` allows to calculate similarities using Euclidean distance while `CosineRecommender` uses (normalized) Cosine similarity.

The idea is to create a model and to fit it to the ratings matrix created before.

In [13]:
model = implicit.nearest_neighbours.ItemItemRecommender(K = 10)
model.fit(ratings)

## Model evaluation

We evaluate the CF models using Hit Ratio (HR). It is commonly used in literature (see the report for references).

For each user we randomly select 100 items: 1 of them is the test interaction and the other 99 are randomly selected among items that has not been rated by that user. Then we calculate the score of the selected user for each of these items using our model, we create a ranking list and $HR=1$ if the test interaction item (i.e. `positive`) is in the top-10, $0$ otherwise.

Basically we have a hit ($HR=1$) when a positive (test) interaction is (almost, in a top-10) correctly detected by the model among many other non-interactions.

In [14]:
# This matrix is used by the function of implicit library. It is only the transpose of ratings matrix
# It is passed as a parameter in order to not re-calculate it at every call of the evaluation function.
user_items = ratings.T.tocsr()

def evaluate_user_k(k, ratings, test, model, user_items):
    
    num_neg = 99
    
    positive = list(test.keys())[k][1]
    negatives = []
    
    items = []
    items.append(positive)
    
    for i in range(num_neg):
        j = np.random.randint(num_items)
        while (k,j) in ratings.keys():
            j = np.random.randint(num_items)
        items.append(j)
    
    # create the scoreboard and keep track of the top-10
    ranklist =  model.rank_items(k, user_items, items)
    ranklist = ranklist[:10]
    
    # calculate whether we have an hit
    hr = 0
    for item in ranklist:
         if item[0] == positive and item[1] > 0:
                hr = 1
                
    return hr   

In this last cell we actually test the model for each user.
We keep track of the elapsed time and we make a ratio between the number of hits and the total number of users.


In [15]:
s = 0
t = time.time()

for i in range(num_users):
    s += evaluate_user_k(i, ratings, test, model, user_items)
    if i%1000 == 0:
        print(i,'users processed')
        
print('Hit Ratio:', s/num_users)
print('Elapsed time:', time.time() - t)

0 users processed
1000 users processed
2000 users processed
3000 users processed
4000 users processed
5000 users processed
6000 users processed
Hit Ratio: 0.16804635761589404
Elapsed time: 135.52697610855103


Notice that this notebook has been used to test models and gain data in order to make a report.
Parameters and functions changes according to which model has been used. For a complete overview of models and parameters please see the report.